In [2]:
import numpy as np
import pandas as pd
from pandas import read_excel
from sqlalchemy import create_engine
import os
import psycopg2 
import io
import psycopg2.extras

In [3]:
file_name = '../data/corridas.xlsx' # change it to the name of your excel file
df = read_excel(file_name)

In [6]:
df

,Date,Distancia,Duration,Average Pace,Average Speed (km/h),Calories Burned,Climb (m)
0,2022-02-13 09:53:12,5.01,34:22,6:51,875.0,3220,147
1,2022-02-06 09:34:34,2.90,33:30,11:34,519.0,1800,93
2,2022-02-05 09:14:42,8.00,54:55,6:52,875.0,5260,240
3,2022-02-02 07:39:37,5.09,35:05,6:53,871.0,3370,172
4,2022-01-30 10:58:47,5.42,37:12,6:52,874.0,3550,164
...,...,...,...,...,...,...,...
390,2015-05-31 08:45:46,17.30,1:55:01,6:39,902.0,13810,266
391,2015-05-24 10:20:25,13.62,1:35:20,7:00,857.0,10800,208
392,2015-05-03 09:44:33,10.89,1:20:27,7:23,812.0,8430,187
393,2015-04-26 10:52:26,11.10,1:12:07,6:30,924.0,8290,165


In [8]:
'00:' + df['Duration'][0]

'00:34:22'

In [12]:
durations = []
for index, row in df.iterrows():
    if len(row['Duration']) <= 5:
        durations.append('00:' + row['Duration'])
    else:
        durations.append(row['Duration'])


In [14]:
df['Duration'] = durations

In [15]:
df

,Date,Distancia,Duration,Average Pace,Average Speed (km/h),Calories Burned,Climb (m)
0,2022-02-13 09:53:12,5.01,00:34:22,6:51,875.0,3220,147
1,2022-02-06 09:34:34,2.90,00:33:30,11:34,519.0,1800,93
2,2022-02-05 09:14:42,8.00,00:54:55,6:52,875.0,5260,240
3,2022-02-02 07:39:37,5.09,00:35:05,6:53,871.0,3370,172
4,2022-01-30 10:58:47,5.42,00:37:12,6:52,874.0,3550,164
...,...,...,...,...,...,...,...
390,2015-05-31 08:45:46,17.30,1:55:01,6:39,902.0,13810,266
391,2015-05-24 10:20:25,13.62,1:35:20,7:00,857.0,10800,208
392,2015-05-03 09:44:33,10.89,1:20:27,7:23,812.0,8430,187
393,2015-04-26 10:52:26,11.10,1:12:07,6:30,924.0,8290,165


In [16]:
DATABASE_URL = os.environ['DATABASE_URL']


In [17]:
df = df[['Date','Distancia','Duration']]

In [18]:
conn = psycopg2.connect(DATABASE_URL, sslmode='require')
if len(df) > 0:
    df_columns = list(df)
    # create (col1,col2,...)
    columns = ",".join(df_columns)

    # create VALUES('%s', '%s",...) one '%s' per column
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

    #create INSERT INTO table (columns) VALUES('%s',...)
    insert_stmt = "INSERT INTO {} ({}) {}".format('corridas',columns,values)

    cur = conn.cursor()
    psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()
    cur.close()

In [34]:
conn.close()